In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import pickle as pkl
from astropy.timeseries import LombScargle
from astropy.timeseries import LombScargleMultiband

/tmp/ipykernel_6727/1123200335.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Periodograms

In [13]:
data_global = pd.read_pickle("../../PeriodogramsGaia/dataset/data_global.pkl")

In [11]:
def periodograms_band(lc,period, d_folder_lc, freq):
    times, fluxs, ferrs, bands = [], [], [], []
    dict_per = {}
    for (name, data) in lc.groupby("band"):
        band = data['band'].iloc[0]
        mask = data["rejected_by_variability"]=="false"
        time, flux, flux_err =data.loc[mask][['time', 'flux', 'flux_error']].values.T
        per = LombScargle(time, flux, flux_err).power(freq)
        dict_per[name] = per
        # multiband
        times += list(time)
        fluxs += list(flux)
        ferrs += list(flux_err)
        bands += [name]*len(flux)

    per = LombScargleMultiband(times, fluxs, bands, ferrs).power(freq)
    dict_per['multiband'] = per
    with open(d_folder_lc + '/periodograms.pkl', 'wb') as handle:
        pkl.dump(dict_per, handle, protocol=pkl.HIGHEST_PROTOCOL)
    

In [20]:
directory = '../dataset'

for folder_type in os.listdir(directory):
    #../dataset/ECL
    d_folder_type = os.path.join(directory, folder_type)
    if folder_type == "ECL":
        freq = np.arange(0.001473, 2*4.999929 , 1e-4)
        with open('../dataset/'+folder_type+'/frequencies_'+folder_type+'.pkl', 'wb') as handle:
            pkl.dump(freq, handle, protocol=pkl.HIGHEST_PROTOCOL)
    elif folder_type == "RR":
        freq = np.arange(1.00061, 2*4.973236 , 1e-4)
        with open('../dataset/'+folder_type+'/frequencies_'+folder_type+'.pkl', 'wb') as handle:
            pkl.dump(freq, handle, protocol=pkl.HIGHEST_PROTOCOL)

    # para que no entre en el dataglobal
    if folder_type.find('.') == -1:
        for folder_lc in os.listdir(d_folder_type):
            # para que no entre en el archivo de las freq
            if folder_lc.find('.') == -1:
                d_folder_lc = os.path.join(d_folder_type, folder_lc)
                lc = pd.read_pickle(os.path.join(d_folder_lc, folder_lc+'.pkl'))
                name_lc = lc.source_id.iloc[0]
                period = data_global[data_global.index == name_lc].values[0][0]
                periodograms_band(lc, period,d_folder_lc, freq)